In [ ]:
# import pandas library as pd

import pandas as pd 
from gammapy.datasets import FluxPointsDataset
from gammapy.catalog import CATALOG_REGISTRY
import os
import sys

format_csv = ".csv"
format_fits = ".fits"

sed_type="e2dnde"
# sed_type="dnde"

df_columns=[]
datasets_fp = []
sources_catalogs = []
  
# create an Empty DataFrame object
df = pd.DataFrame()


def getting_source_data(dict_TeV_counterparts = None, catalogs_tags = None):
    '''
    To write Docstring!!!!
    '''
    
#     LHAASO_range = range(len())
    for LHAASO_name in list(dict_TeV_counterparts.keys()):
#     for i in LHAASO_range:

#         LHAASO_name = dict_key[i]
        LHAASO_txt = LHAASO_name.replace(" ", "")
        
#         source_range = range(len())
        for source_name in dict_TeV_counterparts[LHAASO_name]:
            
            source_txt = source_name.replace(" ", "")

            catalog_source = []

            catalog_fp = []
            
            source_catalog = []

            dataset_fp = []

#             source_name = dict_TeV_counterparts[LHAASO_name][j]
            
#             catalog_range = range(len(catalogs_tags))
#             for k in catalog_range: 
            for catalog_tag in catalogs_tags:    
#                 catalog_tag = catalogs_tags[k]

                catalog = CATALOG_REGISTRY.get_cls(catalog_tag)()
                
                try:
                    
                    src = catalog[source_name]
                    
                    catalog_source.append(catalog_tag)
                    
                    ds_name = f"{source_name}: {catalog_tag}"
                    ds = FluxPointsDataset(
                        models = src.sky_model(),
                        data=src.flux_points, 
                        name=  ds_name     
                    )
                    
                    source_catalog.append(src)
                    dataset_fp.append(ds)
                    catalog_fp.append(catalog_tag)

                    table = ds.data.to_table(
                        sed_type = sed_type, 
                        formatted = True
                    )
                    
                    # Setting the units

                    
                    # Writes the flux points table in the fits format
                    file_name = f'{LHAASO_txt}_{source_txt}_{catalog_tag}{format_fits}'
                    path_os = os.path.abspath(
                        os.path.join(
                            f"{path_flux_points_tables}/{file_name}"
                        )
                    )      
                    
                    if path_os not in sys.path:
                        sys.path.append(path_os)

                    table.write(
                        f"{path_os}",
                        format = 'fits', 
                        overwrite = True
                    )
                    
                    # Writes the flux points table in the csv format
                    file_name = f'{LHAASO_txt}_{source_txt}_{catalog_tag}{format_csv}'
                    path_os = os.path.abspath(
                        os.path.join(
                            f"{path_flux_points_tables}/{file_name}"
                        )
                    )
                     
                    if path_os not in sys.path:
                        sys.path.append(path_os)

                    table.write(
                        f"{path_os}",
                        format = 'ascii.ecsv', 
                        overwrite = True
                    )   
                    
                except:
                    pass

                df_columns_source = [LHAASO_name, LHAASO_txt, source_name, source_txt, ', '.join(catalog_source), ', '.join(catalog_fp)]
            df_columns.append(df_columns_source)
            datasets_fp.append(dataset_fp)
            sources_catalogs.append(source_catalog)

    df = pd.DataFrame(df_columns, columns = ['LHAASO_name', "LHAASO_txt",'source_name', "source_txt", 'catalog', 'flux_points']) 
    df_fp = df[df["flux_points"].str.len() != 0]
    df_fp = df_fp.reset_index(drop = True)
    df_fp.index.name = 'source_index'
    
    file_name = "data_frame_flux_points.csv"
    path_file = f"{os.getcwd()}/{file_name}"
    df_fp.to_csv(path_file, index = "True" )
    
    df_no_fp = df[df["flux_points"].str.len() == 0]
    df_no_fp = df_no_fp.reset_index(drop = True)
    df_no_fp.index.name = 'source_index'
    
    file_name = "data_frame_no_flux_points.csv"
    path_file = f"{os.getcwd()}/{file_name}"
    df_no_fp.to_csv(path_file, index = "True")
    print(f"Sources without flux points table: {list(df_no_fp['source_name'])}")
    
    sources_catalogs[:] = list(filter(None, sources_catalogs))
    datasets_fp[:] = list(filter(None, datasets_fp))
    
    return df_fp, datasets_fp, sources_catalogs